In [12]:
import requests
from bs4 import BeautifulSoup
import lxml
import re
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd

pd.set_option('display.max_colwidth', 200)
%matplotlib inline

In [17]:
pmcids = []
for i in tqdm(range(1, 4)):
    URL = f"https://europepmc.org/search?query=%28ANNOTATION_PROVIDER%3A%22DisGeNET%22%20AND%20ANNOTATION_PROVIDER%3A%22Europe%20PMC%22%20AND%20ANNOTATION_PROVIDER%3A%22HES-SO_SIB%22%20AND%20ANNOTATION_PROVIDER%3A%22Metagenomics%22%20AND%20ANNOTATION_PROVIDER%3A%22OntoGene%22%20AND%20ANNOTATION_PROVIDER%3A%22OpenTargets%22%20AND%20ANNOTATION_PROVIDER%3A%22PubTator_NCBI%22%29&page={i}?javascript_support=no"
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, 'html.parser')
    pmcids_ = re.findall(r'PMC\d+', str(soup))

    pmcids.extend(pmcids_)  # Use extend to flatten the list

# pmcids will now contain all PMCIDs extracted

100%|█████████████████████████████████████████████| 3/3 [00:08<00:00,  2.91s/it]


In [20]:
pmcids

['PMC5810984',
 'PMC5774238',
 'PMC5671944',
 'PMC5559326',
 'PMC4902368',
 'PMC4766352',
 'PMC4877170',
 'PMC4735314',
 'PMC4713070',
 'PMC4682834',
 'PMC4690073',
 'PMC4593004',
 'PMC4539664',
 'PMC4588721',
 'PMC4567394',
 'PMC4471120',
 'PMC4539360',
 'PMC4457031',
 'PMC4460277',
 'PMC4454778',
 'PMC4363147',
 'PMC6680266',
 'PMC4583332',
 'PMC4151597',
 'PMC4747096',
 'PMC5810984',
 'PMC5774238',
 'PMC5671944',
 'PMC5559326',
 'PMC4902368',
 'PMC4766352',
 'PMC4877170',
 'PMC4735314',
 'PMC4713070',
 'PMC4682834',
 'PMC4690073',
 'PMC4593004',
 'PMC4539664',
 'PMC4588721',
 'PMC4567394',
 'PMC4471120',
 'PMC4539360',
 'PMC4457031',
 'PMC4460277',
 'PMC4454778',
 'PMC4363147',
 'PMC6680266',
 'PMC4583332',
 'PMC4151597',
 'PMC4747096',
 'PMC5810984',
 'PMC5774238',
 'PMC5671944',
 'PMC5559326',
 'PMC4902368',
 'PMC4766352',
 'PMC4877170',
 'PMC4735314',
 'PMC4713070',
 'PMC4682834',
 'PMC4690073',
 'PMC4593004',
 'PMC4539664',
 'PMC4588721',
 'PMC4567394',
 'PMC4471120',
 'PMC45393

In [44]:
import requests
from bs4 import BeautifulSoup
import lxml
from collections import defaultdict
import re
import spacy

# Load SpaCy model
nlp = spacy.load("en_core_sci_sm")


titleMapsBody = {
    'INTRO': ['introduction', 'background', 'related literature', 'literature review', 'objective', 'aim ', 'purpose of this study', 'study (purpose|aim|aims)', '(\d)+\. (purpose|aims|aim)', '(aims|aim|purpose) of the study', '(the|drug|systematic|book) review', 'review of literature', 'related work', 'recent advance'],
    'METHODS': ['supplement', 'methods and materials', 'method', 'material', 'experimental procedure', 'implementation', 'methodology', 'treatment', 'statistical analysis', "experimental", '(\d)+\. experimental$', 'experimental (section|evaluation|design|approach|protocol|setting|set up|investigation|detail|part|pespective|tool)', "the study", '(\d)+\. the study$', "protocol", "protocols", 'study protocol', 'construction and content', 'experiment (\d)+', '^experiments$', 'analysis', 'utility', 'design', '(\d)+\. theory$', "theory", 'theory and ', 'theory of '],
    'RESULTS': ['result', 'finding', 'diagnosis'],
    'DISCUSS': ['discussion', 'management of', '(\d)+\. management', 'safety and tolerability', 'limitations', 'perspective', 'commentary', '(\d)+\. comment'],
    'CONCL': ['conclusion', 'key message', 'future', 'summary', 'recommendation', 'implications for clinical practice','concluding remark'],
    'CASE': ['case study report', 'case report', 'case presentation', 'case description', 'case (\d)+', '(\d)+\. case', 'case summary', 'case history'],
    'ACK_FUND': ['funding', 'acknowledgement', 'acknowledgment', 'financial disclosure'],
    'AUTH_CONT': ['author contribution', 'authors\' contribution', 'author\'s contribution'],
    'COMP_INT': ['competing interest', 'conflict of interest', 'conflicts of interest', 'disclosure', 'decleration'],
    'ABBR': ['abbreviation'],
    'SUPPL': ['supplemental data', 'supplementary file', 'supplemental file', 'supplementary data', 'supplementary figure', 'supplemental figure', 'supporting information', 'supplemental file', 'supplemental material', 'supplementary material', 'supplement material', 'additional data files', 'supplemental information', 'supplementary information', 'supplemental information', 'supporting information', 'supplemental table', 'supplementary table', 'supplement table', 'supplementary material', 'supplemental material', 'supplement material', 'supplementary video']
}

titleExactMapsBody = {
    'INTRO': ["aim", "aims", "purpose", "purposes", "purpose/aim", "purpose of study", "review", "reviews", "minireview"],
    'METHODS': ["experimental", "the study", "protocol", "protocols"],
    'DISCUSS': ["management", "comment", "comments"],
    'CASE': ["case", "cases"]
}

titleMapsBack = {
    'REF': ['reference', 'literature cited', 'references', 'bibliography'],
    'ACK_FUND': ['funding', 'acknowledgement', 'acknowledgment', 'aknowledgement', 'acknowlegement', 'open access', 'financial support', 'grant', 'author note', 'financial disclosure'],
    'ABBR': ['abbreviation', 'glossary'],
    'COMP_INT': ['competing interest', 'conflict of interest', 'conflicts of interest', 'disclosure', 'decleration', 'conflits', 'interest'],
    'SUPPL': ['supplementary', 'supporting information', 'supplemental', 'web extra material'],
    'APPENDIX': ['appendix', 'appendices'],
    'AUTH_CONT': ['author', 'contribution']
}


def categorize_section(title):
    # Lowercase the title for consistency
    title_lower = title.lower()

    # Check if title exactly matches any pattern in titleExactMapsBody
    for category, patterns in titleExactMapsBody.items():
        if title_lower in patterns:
            return category

    # Check if title matches any pattern in titleMapsBody
    for category, patterns in titleMapsBody.items():
        for pattern in patterns:
            if re.search(pattern, title_lower):
                return category

    # Check if title matches any pattern in titleMapsBack
    for category, patterns in titleMapsBack.items():
        for pattern in patterns:
            if re.search(pattern, title_lower):
                return category

    # Default category if no match found
    return 'OTHER'



def extract_sections_and_sentences(soup):
    sections_sentences = defaultdict(list)

    # Iterate through each section in the document
    for sec in soup.find_all('sec'):
        # Extract section title
        title = sec.find('title')
        if title:
            title_text = title.get_text().strip()
            section_type = categorize_section(title_text)

            # Extract sentences from paragraphs in the section
            for p in sec.find_all('p'):
                paragraph_text = p.get_text().strip()
                doc = nlp(paragraph_text)
                sentences = [sent.text.strip() for sent in doc.sents]
                sections_sentences[section_type].extend(sentences)

    return sections_sentences

In [45]:
# # Main execution flow
# pmcid = 'PMC1501050'
# url = f"https://www.ebi.ac.uk/europepmc/webservices/rest/{pmcid}/fullTextXML"

# try:
#     response = requests.get(url)
#     if response.status_code == 200:
#         xml_content = response.content
#         soup = BeautifulSoup(xml_content, 'lxml')

#         # Extract sections and sentences
#         sections_sentences = extract_sections_and_sentences(soup)

#         # The dictionary 'sections_sentences' now contains the extracted data
#         # You can add additional processing or output logic here
#     else:
#         print(f"Failed to retrieve data: HTTP Status Code {response.status_code}")
# except requests.RequestException as e:
#     print(f"Request failed: {e}")



In [49]:
import random


# Function to extract the largest and second largest words from a sentence
def get_largest_words(sentence):
    words = sentence.split()
    if len(words) > 1:
        sorted_words = sorted(words, key=len, reverse=True)
        return sorted_words[0], sorted_words[1]
    elif words:
        return words[0], None
    return None, None

# Function to generate fake data for a given PMCID
def generate_fake_data(pmcid, sections_sentences):
    fake_data = {
        "src": "PMC",
        "id": pmcid,
        "provider": "OpenTargets",
        "anns": []
    }

    annotation_types = ["Gene Disease Relationship", "Gene Drug Relationship", "Disease Drug Relationship"]
    # Select 10 largest sentences from all sentences
    all_sentences = [sentence for sentences in sections_sentences.values() for sentence in sentences]
    largest_sentences = sorted(all_sentences, key=len, reverse=True)[:10]

    for annotation_type in annotation_types:
        num_sentences = random.randint(1, 10)  # Random number of sentences between 1 and 10
        selected_sentences = random.sample(largest_sentences, min(num_sentences, len(largest_sentences)))

        for sentence in selected_sentences:
            largest_word, second_largest_word = get_largest_words(sentence)
            section = [sec for sec, sentences in sections_sentences.items() if sentence in sentences][0]

            annotation = {
                "type": annotation_type,
                "exact": sentence,
                "section": section,
                "tags": []
            }

            if largest_word:
                annotation["tags"].append({
                    "name": largest_word,
                    "uri": "http://purl.uniprot.org/uniprot/Q13228"  # Fake URI
                })

            if second_largest_word:
                annotation["tags"].append({
                    "name": second_largest_word,
                    "uri": "http://purl.uniprot.org/uniprot/Q13228"  # Fake URI
                })

            fake_data["anns"].append(annotation)

    return fake_data



In [51]:
import json

# File to write the JSON Lines
output_file_path = '../data/fake_data.jsonl'

with open(output_file_path, 'w') as outfile:
    for pmcid in tqdm(pmcids, desc="Processing PMCIDs"):
        url = f"https://www.ebi.ac.uk/europepmc/webservices/rest/{pmcid}/fullTextXML"
        try:
            response = requests.get(url)
            if response.status_code == 200:
                xml_content = response.content
                soup = BeautifulSoup(xml_content, 'lxml')

                # Extract sections and sentences
                sections_sentences = extract_sections_and_sentences(soup)

                # Generate fake data for the PMCID
                fake_data = generate_fake_data(pmcid, sections_sentences)

                # Write the fake data as a JSON line
                json_line = json.dumps(fake_data) + "\n"
                outfile.write(json_line)

            else:
                print(f"Failed to retrieve data for PMCID {pmcid}: HTTP Status Code {response.status_code}")
        except requests.RequestException as e:
            print(f"Request failed for PMCID {pmcid}: {e}")

print("Processing completed. Fake data written to", output_file_path)


Processing PMCIDs:   3%|▋                        | 2/75 [00:02<01:06,  1.09it/s]

Failed to retrieve data for PMCID PMC5774238: HTTP Status Code 404


Processing PMCIDs:   5%|█▎                       | 4/75 [00:05<01:34,  1.33s/it]

Failed to retrieve data for PMCID PMC5559326: HTTP Status Code 404


Processing PMCIDs:   9%|██▎                      | 7/75 [00:10<01:33,  1.37s/it]

...

Failed to retrieve data for PMCID PMC4877170: HTTP Status Code 404


Processing PMCIDs:  87%|████████████████████▊   | 65/75 [02:05<00:17,  1.78s/it]

Failed to retrieve data for PMCID PMC4567394: HTTP Status Code 404


Processing PMCIDs: 100%|████████████████████████| 75/75 [02:27<00:00,  1.97s/it]

Failed to retrieve data for PMCID PMC4747096: HTTP Status Code 404
Processing completed. Fake data written to ../data/fake_data.jsonl
